In [1]:
from PIL import Image
import os
import matplotlib.pyplot as plt
from array import array
import numpy as np
import cv2
import sift
from scipy import ndimage
import warp
import homography

In [ ]:
fp = np.array([[0, 1, 2],[4, 5, 6],[1, 1, 1]])

m = np.mean(fp[:2], axis=1)
print(m)
maxstd = max(np.std(fp[:2], axis=1)) + 1e-9
print(maxstd)
C1 = np.diag([1/maxstd, 1/maxstd, 1])
print(C1)
C1[0][2] = -m[0]/maxstd
C1[1][2] = -m[1]/maxstd
print(C1)
fp = np.dot(C1, fp)
print(fp)

In [4]:
from scipy import ndimage

In [ ]:
im = np.array(Image.open("/home/jaxa/toshiki/sfmtutorial/scan9_mini/00000016.jpg").convert("L"))
H = np.array([[1.4, 0.05, -100], [0.05, 1.5, -100], [0, 0, 1]])
im2 = ndimage.affine_transform(im, H[:2,:2], (H[0,2],H[1,2]))

plt.figure()
plt.gray()
plt.imshow(im2)
plt.show()

In [ ]:
im1 = np.array(Image.open("/home/jaxa/toshiki/sfmtutorial/IMG20241217171723.jpg").convert("L"))
im2 = np.array(Image.open("/home/jaxa/toshiki/sfmtutorial/scan9_mini/00000016.jpg").convert("L"))

tp = np.array([[264, 538, 540, 264], [140, 36, 665, 805], [1, 1, 1, 1]])
im3 = warp.image_in_image(im1, im2, tp)

plt.figure()
plt.gray()
plt.imshow(im3)
plt.show()

In [ ]:
im1 = np.array(Image.open("/home/jaxa/toshiki/sfmtutorial/IMG20241217171723.jpg").convert("L"))
im2 = np.array(Image.open("/home/jaxa/toshiki/sfmtutorial/scan9_mini/00000016.jpg").convert("L"))

tp = np.array([[264, 538, 540, 264], [140, 36, 665, 805], [1, 1, 1, 1]])

m, n = im1.shape[:2]
fp = np.array([[0,m,m,0],[0,0,n,n],[1,1,1,1]])

tp2 = tp[:, :3]
fp2 = fp[:, :3]

H = homography.Haffine_from_points(tp2, fp2)
im1_t = ndimage.affine_transform(im1, H[:2,:2], (H[0,2],H[1,2]), im2.shape[:2])

alpha = warp.alpha_for_triangle(tp2, im2.shape[0], im2.shape[1])
im3 = (1-alpha)*im2 + alpha*im1_t

tp2 = tp[:, [0,2,3]]
fp2 = fp[:, [0,2,3]]

H = homography.Haffine_from_points(tp2, fp2)
im1_t = ndimage.affine_transform(im1, H[:2,:2], (H[0,2],H[1,2]), im2.shape[:2])

alpha = warp.alpha_for_triangle(tp2, im2.shape[0], im2.shape[1])
im4 = (1-alpha)*im3 + alpha*im1_t

plt.figure()
plt.gray()
plt.imshow(im4)
plt.show()

In [ ]:
dir_path = "room"
out_path = "room_resize"
out_path = os.path.join(dir_path, out_path)

files = os.listdir(dir_path)
files = files[:-1]
print(files)

for i in files:
    img_path = os.path.join(dir_path, i)
    img = Image.open(img_path)
    img_resized = img.resize((1600, 1200))
    output_path = os.path.join(out_path, i)
    img_resized.save(output_path, quality=90)



In [ ]:
images_path = "/home/jaxa/toshiki/sfmtutorial/room/room_resize"
sift_path = "/home/jaxa/toshiki/sfmtutorial/room/room_sift"

sift.process_image_multiview(images_path, sift_path)
matchscores = sift.matching_multiview(images_path, sift_path)

In [ ]:
np.save("matchscores_room.npy", matchscores)

In [6]:
matchscores = np.load("matchscores_room.npy")
print(matchscores)
print(matchscores.shape)

[[5.396e+03 4.000e+00 6.470e+02 2.140e+02 1.130e+03 6.120e+02 5.600e+01]
 [4.000e+00 2.067e+03 7.700e+01 1.000e+00 0.000e+00 1.000e+00 1.180e+02]
 [6.470e+02 7.700e+01 3.880e+03 1.100e+01 5.590e+02 3.230e+02 2.000e+02]
 [2.140e+02 1.000e+00 1.100e+01 7.319e+03 3.960e+02 1.026e+03 0.000e+00]
 [1.130e+03 0.000e+00 5.590e+02 3.960e+02 5.908e+03 1.078e+03 1.000e+01]
 [6.120e+02 1.000e+00 3.230e+02 1.026e+03 1.078e+03 6.389e+03 0.000e+00]
 [5.600e+01 1.180e+02 2.000e+02 0.000e+00 1.000e+01 0.000e+00 1.220e+03]]
(7, 7)


In [8]:
sift_path = "/home/jaxa/toshiki/sfmtutorial/room/room_sift"
files = os.listdir(sift_path)

l = {}
d = {}

for i in range(len(files)):
    l[i], d[i] = sift.read_features_from_file(os.path.join(sift_path, files[i]))
    l[i][:,[0,1]] = l[i][:,[1,0]]

matches = {}

for i in range(len(files)-1):
    matches[i] = sift.match(d[i+1],d[i])

In [10]:
print(matches)


{0: array([0, 0, 0, ..., 0, 0, 0]), 1: array([0, 0, 0, ..., 0, 0, 0]), 2: array([0, 0, 0, ..., 0, 0, 0]), 3: array([2412,    0,    0, ...,    0,    0,    0]), 4: array([0, 0, 0, ..., 0, 0, 0]), 5: array([0, 0, 0, ..., 0, 0, 0])}


In [11]:
def convert_points(matchscores, j):
    ndx = matchscores[j].nonzero()[0]
    fp = homography.make_homog(l[j+1][ndx,:2].T)
    ndx2 = [int(matchscores[j][i]) for i in ndx]
    tp = homography.make_homog(l[j][ndx2,:2].T)

    return fp, tp

In [16]:
model = homography.RansacModel()

fp, tp = convert_points(matches, 0)
H_01 = homography.H_from_ransac(fp, tp, model)[0]

fp, tp = convert_points(matches, 1)
H_12 = homography.H_from_ransac(fp, tp, model)[0]

fp, tp = convert_points(matches, 2)
# H_32 = homography.H_from_ransac(fp, tp, model)[0]

fp, tp = convert_points(matches, 3)
H_43 = homography.H_from_ransac(fp, tp, model)[0]



ValueError: did not meet fit acceptance criteria